

**В данном кейсе рассчитаны следующие метрики:**
* retention
* rolling retention
* lifetime
* churn rate
* mau
* wau
* dau

**Исходные данные**  
Файлами для работы являются registrations.csv и entries.csv. В них хранятся данные о регистрациях пользователей и входа на платформу соответственно.

In [1]:
# готовим датафрейм для расчета метрик
import pandas as pd
import warnings
warnings.simplefilter(action = 'ignore', category = )
registrations = pd.read_csv('registrations.csv', sep = ';')
entries = pd.read_csv('entries.csv', sep = ';')
df = pd.merge(registrations, entries, how = 'left', on = 'user_id') # объединенный датафрейм для расчета метрик
df['entry_date'] = pd.to_datetime(df['entry_date'], format = '%Y-%m-%d')
df['registration_date'] = pd.to_datetime(df['registration_date'], format = '%Y-%m-%d')

In [3]:
df.head()

,user_id,registration_date,entry_date
0,1,2021-01-01,2021-01-01
1,1,2021-01-01,2021-01-03
2,1,2021-01-01,2021-01-04
3,1,2021-01-01,2021-01-04
4,1,2021-01-01,2021-01-05


**1. Посчитайте Retention 15 дня для пользователей, зарегистрированных в январе**  
Cохраните результат в переменную retention_15_day.  
Примечание: результат округлите до 5 знаков после запятой.

In [4]:
df['diff'] = df['entry_date'] - df['registration_date']
df_jan = df[df['registration_date'].dt.month == 1]
df_jan_15 = df_jan[df['diff'] == '15 days']
retention_15_day = round(df_jan_15['user_id'].nunique()/df_jan['user_id'].nunique() * 100, 5)
retention_15_day

C:\Users\Анна\AppData\Local\Temp\ipykernel_10572\3973618502.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_jan_15 = df_jan[df['diff'] == '15 days']


54.65116

**2. Посчитайте Rolling-retention 30 дня для пользователей из той же когорты**  
Сохраните результат в переменную rolling_retention.  
Примечание: результат округлите до 5 знаков после запятой.

In [5]:
df_jan_roll_30 = df_jan[df['diff'] >= '30 days']
rolling_retention = round(df_jan_roll_30['user_id'].nunique()/df_jan['user_id'].nunique() * 100, 5)
rolling_retention

C:\Users\Анна\AppData\Local\Temp\ipykernel_10572\1622781401.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_jan_roll_30 = df_jan[df['diff'] >= '30 days']


29.06977

**3. Посчитайте Lifetime по всем пользователям, посчитанный как интеграл от n-day retention**  
Сохраните результат в переменную lifetime  
Примечание: результат округлите до 5 знаков после запятой

In [6]:
df_lifetime = df.groupby(['diff']).agg({'user_id': 'nunique'}).reset_index()
df_lifetime['n_day_ret'] = df_lifetime['user_id'] / df['user_id'].nunique()
lifetime = round(sum(df_lifetime['n_day_ret']), 5)
lifetime

14.804

**4. Посчитайте Churn rate 29 дня, посчитанный по всем пользователям**  
Сохраните результат в переменную churn_29

In [7]:
df_29 = df[df['diff'] >= '29 days']
churn_29 = (df['user_id'].nunique() - df_29['user_id'].nunique())/df['user_id'].nunique()
churn_29

0.509

**5. Посчитайте Mau, Wau, Dau за последний месяц/неделю/день записей**  
Сохраните результат в переменные dec_mau, dec_wau, dec_dau соответственно.  
Примечание: последний месяц записей - декабрь. Поэтому mau рассчитываем для декабря (2021 года), для wau берем последнюю неделю - с 25 по 31 декабря, и для dau соответственно последний день - 31 декабря.

In [8]:
df_dec_mau = df[df['entry_date'].dt.month == 12]
dec_mau = df_dec_mau['user_id'].nunique()
dec_mau

133

In [9]:
df_dec_wau = df[(df['entry_date'] >= '2021-12-25') & (df['entry_date'] <= '2021-12-31')]
dec_wau = df_dec_wau['user_id'].nunique()
dec_wau

84

In [10]:
df_dec_dau = df[df['entry_date'] == '2021-12-31']
dec_dau = df_dec_dau['user_id'].nunique()
dec_dau

47

**6. Посчитайте Mau, Wau, Dau усредненные**  
Сохраните результат в переменные avg_dec_mau, avg_dec_wau, avg_dec_dau соответственно.  
Примечание: результаты округлите до 5 знаков после запятой

In [11]:
df['year_month'] = df['entry_date'].dt.to_period('M')
df_avg_dec_mau = df.groupby(['year_month']).agg({'user_id': 'nunique'}).reset_index()
avg_dec_mau = round(df_avg_dec_mau['user_id'].mean(), 5)
avg_dec_mau

102.58333

In [12]:
df['week'] = df['entry_date'].dt.isocalendar().week
df_avg_dec_wau = df.groupby(['week']).agg({'user_id': 'nunique'}).reset_index()
avg_dec_wau = round(df_avg_dec_wau['user_id'].mean(), 5)
avg_dec_wau

89.86792

In [12]:
df_avg_dec_dau = df.groupby(['entry_date']).agg({'user_id': 'nunique'}).reset_index()
avg_dec_dau = round(df_avg_dec_dau['user_id'].mean(), 5)
avg_dec_dau

40.5589